In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import glob

dir='/Users/lexihenny/Downloads/'
dir_data='/Users/lexihenny/'

In [ ]:
#####PRECIP
yrs=np.arange(1959,2023,1)
#yrs=np.arange(2022,2023,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    days_year=365
    if year%4==0:
        days_year=366
        
    dates_year=[dt.datetime(year,1,1)+dt.timedelta(days=x) for x in range(days_year)]

    files=glob.glob(dir_data+'fcst_phy2m.061_tprat.reg_tl319.'+str(year)+'*')
    files=[x for x in files if 'idx' not in x]
    files=sorted(files)

    count=0
    for file in files:
        ds=xr.open_dataset(file,engine='cfgrib')
        field=ds['tpratsfc']
        
        lons_west=[x for x in ds.longitude.values if x<180]
        lons_east=[x for x in ds.longitude.values if x>=180]
        field_west=field.sel(longitude=lons_west)
        field_east=field.sel(longitude=lons_east)
        field_east['longitude']=[x-360. for x in lons_east]
        field=xr.concat([field_east,field_west],dim='longitude')

        if count==0:
            field_concat=field
            count=1
        else:
            field_concat=xr.concat([field_concat,field],dim='time')
    
    count=0
    for date in dates_year:
        date_range=[date+dt.timedelta(hours=6*x) for x in range(4)]
        field_sel=field_concat.sel(time=date_range)
        field_mean=field_sel.mean(dim=('time','step'),skipna=True)
        if count==0:
            precip_concat=field_mean
            count=1
        else:
            precip_concat=xr.concat([precip_concat,field_mean],dim='time')
            
            
    precip_concat['time']=dates_year
    
    dk=xr.Dataset()
    dk['precip']=(('time','lat','lon'),precip_concat.values)
    dk.coords['time']=precip_concat.time
    dk.coords['lat']=precip_concat.latitude.values
    dk.coords['lon']=precip_concat.longitude.values
    dk['precip'].attrs["units"]='mm*day-1'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    
    try:
        os.remove(dir_data+'jra55_precip_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir_data+'jra55_precip_'+str(year)+'.nc',mode='w',format='NETCDF4')
    #sys.exit()

In [ ]:
dk=xr.open_dataset(dir_data+'jra55_precip_1958.nc')
test=dk['precip'][1,:,:]

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=50,cmap=plt.cm.RdYlGn_r)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
####IVT, etc.
yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    days_year=365
    if year%4==0:
        days_year=366
    
    #files=glob.glob(dir_data+'anl_column.157_uwv.reg_tl319.'+str(year)+'*')
    #files=glob.glob(dir_data+'anl_column.152_vwv.reg_tl319.'+str(year)+'*')
    #files=glob.glob(dir_data+'anl_p125.033_ugrd.'+str(year)+'*')
    files=glob.glob(dir_data+'anl_p125.034_vgrd.'+str(year)+'*')
    
    files=[x for x in files if 'idx' not in x]
    files=sorted(files)

    count=0
    for file in files:
        ds=xr.open_dataset(file,engine='cfgrib')
        #field=ds['viwve']
        #field=ds['viwvn']
        #field=ds['u'].sel(isobaricInhPa=850)
        field=ds['v'].sel(isobaricInhPa=850)
        
        lons_west=[x for x in ds.longitude.values if x<180]
        lons_east=[x for x in ds.longitude.values if x>=180]
        field_west=field.sel(longitude=lons_west)
        field_east=field.sel(longitude=lons_east)
        field_east['longitude']=[x-360. for x in lons_east]
        field=xr.concat([field_east,field_west],dim='longitude')

        if count==0:
            field_concat=field
            count=1
        else:
            field_concat=xr.concat([field_concat,field],dim='time')
    
    dk=xr.Dataset()
    dk['v']=(('time','lat','lon'),field_concat.values)
    dk.coords['time']=field_concat.time
    dk.coords['lat']=field_concat.latitude.values
    dk.coords['lon']=field_concat.longitude.values
    dk['v'].attrs["units"]='m*s-1'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    dk['v'].attrs["name"]='northward wind speed'
    
    try:
        os.remove(dir_data+'jra55_v850_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir_data+'jra55_v850_'+str(year)+'.nc',mode='w',format='NETCDF4')
    
    for file in files:
        os.remove(file)

In [ ]:
yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir_data+'jra55_ivtu_'+str(year)+'.nc')
    ds1=xr.open_dataset(dir_data+'jra55_ivtv_'+str(year)+'.nc')
    ivt=np.sqrt(ds['ivtu']**2+ds1['ivtv']**2)
    
    dk=xr.Dataset()
    dk['ivt']=(('time','lat','lon'),ivt.values)
    dk.coords['time']=ivt.time
    dk.coords['lat']=ivt.lat.values
    dk.coords['lon']=ivt.lon.values
    dk['ivt'].attrs["units"]='kg*m-1*s-1'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    dk['ivt'].attrs["name"]='IVT magnitude'
    
    try:
        os.remove(dir_data+'jra55_ivt_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir_data+'jra55_ivt_'+str(year)+'.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'jra55_v850_'+str(1978)+'.nc')
test=ds['v'][60,:,:]

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=20,cmap=plt.cm.Greys)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
####IVT, etc.
yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    days_year=365
    if year%4==0:
        days_year=366
    
    #files=glob.glob(dir_data+'anl_column.157_uwv.reg_tl319.'+str(year)+'*')
    #files=glob.glob(dir_data+'anl_column.152_vwv.reg_tl319.'+str(year)+'*')
    files=glob.glob(dir_data+'anl_column.054_pwat.reg_tl319.'+str(year)+'*')
    
    files=[x for x in files if 'idx' not in x]
    files=sorted(files)

    count=0
    for file in files:
        ds=xr.open_dataset(file,engine='cfgrib')
        field=ds['tcwv']
        
        lons_west=[x for x in ds.longitude.values if x<180]
        lons_east=[x for x in ds.longitude.values if x>=180]
        field_west=field.sel(longitude=lons_west)
        field_east=field.sel(longitude=lons_east)
        field_east['longitude']=[x-360. for x in lons_east]
        field=xr.concat([field_east,field_west],dim='longitude')

        if count==0:
            field_concat=field
            count=1
        else:
            field_concat=xr.concat([field_concat,field],dim='time')
    
    dk=xr.Dataset()
    dk['iwv']=(('time','lat','lon'),field_concat.values)
    dk.coords['time']=field_concat.time
    dk.coords['lat']=field_concat.latitude.values
    dk.coords['lon']=field_concat.longitude.values
    dk['iwv'].attrs["units"]='kg*m-2'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    dk['iwv'].attrs["name"]='IWV'
    
    try:
        os.remove(dir_data+'jra55_iwv_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir_data+'jra55_iwv_'+str(year)+'.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'jra55_iwv_'+str(1997)+'.nc')
print(ds)
test=ds['iwv'][500,:,:]

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=60,cmap=plt.cm.Greens)
ax.coastlines(resolution='10m')
plt.show()